# Data Preparation

This notebook details the initial steps of preparing the [Vehicles.csv](https://www.kaggle.com/datasets/austinreese/craigslist-carstrucks-data) dataset for analysis. It includes data loading, understanding, and cleaning processes to ensure the data is in a format for further analysis. Key tasks involve handling missing values, encoding categorical variables, normalizing numerical features, and addressing any data quality issues.

By the end of this notebook, the dataset will be transformed into a refined version ready for exploratory data anlanlysis and model development.

### Loading Tools and Data

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer, KNNImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:,.2f}'.format)

df = pd.read_csv('../data/vehicles.csv')

## Understanding The Data
- Dataframe `shape`
- `info`
- `head` and `tail`
- `describe`
- `unique` values

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df

In [ ]:
df.describe()

In [ ]:
unique_check = df.drop(['url', 'region', 'region_url', 'image_url', 'description'], axis=1).copy()
for column, rows in unique_check.items():
    print('----------')
    print(f'{column} --- {df[column].unique()} --- {df[column].dtype}')

## Data Preparation
- Dropping irrelevant columns and rows
- Identifying duplicated columns / incorrect datatypes
- Impute `NaN`
- Renaming columns
- Feature creation or addressing any concerns

In [ ]:
for column, row in df.items():
    print(f'{column} - {round((df[column].isna().sum() / len(df)) * 100,2)}%')
print('--------')
print(f'Shape: {df.shape}')
print(df.isna().sum())

In [ ]:
df = df[[
    # 'id', 'url', 'region', 'region_url', 'VIN', 'size', 
    # 'county', 'image_url', 'description', 'posting_date',
    'price', 'year', 'manufacturer', 'model', 'condition', 'cylinders', 'fuel', 
    'odometer', 'title_status', 'transmission', 'drive', 'type', 'paint_color',
    'state', 'lat', 'long']].copy()

assert len(df.columns) == 16

In [ ]:
# numerical = ['year','odometer','lat','long']
# nominal = ['manufacturer','model','fuel','transmission','type','paint_color','state']
# ordinal = ['condition','cylinders','title_status','drive']

# def impute_nans(df, column):
#     df_notna = df[df[column].notnull()]
#     df_isna = df[df[column].isna()]

#     print(f'processing column: {column}')

#     X_train = df_notna.drop(column, axis=1)
#     y_train = df_notna[column]
#     X_test = df_isna.drop(column, axis=1)

#     numerical_features = [feature for feature in numerical if feature != column]
#     nominal_features = [feature for feature in nominal if feature != column]
#     ordinal_features = [feature for feature in ordinal if feature != column]

#     impute_preprocessor = ColumnTransformer(
#         transformers = [
#             ('numerical', StandardScaler(), numerical_features),
#             ('nominal', OneHotEncoder(sparse_output=False), nominal_features),
#             ('ordinal', OrdinalEncoder(), ordinal_features)])

#     impute_pipeline = Pipeline(
#         steps = [
#             ('preprocessor', impute_preprocessor),
#             ('model', KNNImputer(n_neighbors=10))])

#     print("X_train columns:", X_train.columns)
#     print("X_test columns:", X_test.columns)
#     print("y_train:", y_train.name)

#     impute_pipeline.fit(X_train, y_train)
#     y_pred = impute_pipeline.predict(X_test)
#     df.loc[df[column].isna(), column] = y_pred
        
#     return df

# nan_columns = df.columns[df.isna().any()]
# for column in nan_columns:
#     df = impute_nans(df, column)

# df.head()

In [ ]:
dataframes = np.array_split(df, 100)
for i, smaller_df in enumerate(dataframes):
        smaller_df.to_csv(f'../data/vehicles_part_{i+1}.csv', index=False)

file_path = '../data/vehicles.csv'
        
def process_dataframe(df):
    df = pd.read_csv(file_path)
    df = df[[
        'price', 'year', 'manufacturer', 'model', 'condition', 'cylinders', 'fuel', 
        'odometer', 'title_status', 'transmission', 'drive', 'type', 'paint_color',
        'state', 'lat', 'long']].copy()
    
    assert len(df.columns) == 16

    numerical = ['year','odometer','lat','long']
    nominal = ['manufacturer','model','fuel','transmission','type','paint_color','state']
    ordinal = ['condition','cylinders','title_status','drive']

    def impute_nans(df, column):
        df_notna = df[df[column].notnull()]
        df_isna = df[df[column].isna()]

        print(f'processing column: {column}')

        X_train = df_notna.drop(column, axis=1)
        y_train = df_notna[column]
        X_test = df_isna.drop(column, axis=1)

        numerical_features = [feature for feature in numerical if feature != column]
        nominal_features = [feature for feature in nominal if feature != column]
        ordinal_features = [feature for feature in ordinal if feature != column]

        impute_preprocessor = ColumnTransformer(
            transformers = [
                ('numerical', StandardScaler(), numerical_features),
                ('nominal', OneHotEncoder(sparse_output=False), nominal_features),
                ('ordinal', OrdinalEncoder(), ordinal_features)])

        impute_pipeline = Pipeline(
            steps = [
                ('preprocessor', impute_preprocessor),
                ('model', IterativeImputer(estimator=RandomForestRegressor(), random_state=42))])
        
        print("X_train columns:", X_train.columns)
        print("X_test columns:", X_test.columns)
        print("y_train:", y_train.name)

        impute_pipeline.fit(X_train, y_train)
        y_pred = impute_pipeline.predict(X_test)
        df.loc[df[column].isna(), column] = y_pred

        return df

    nan_columns = df.columns[df.isna().any()]
    for column in nan_columns:
        df = impute_nans(df, column)

    return df

file_paths = [f'../data/vehicles_part_{i}.csv' for i in range(1, 101)]

for file_path in file_paths:
    processed_df = process_dataframe(file_path)
    processed_df.to_csv(file_path, index=False)  # Save to the same file path

In [ ]:
# numerical = ['DurationMonths', 'MonthlyCharges']
# categorical = ['Contract']
# estimator = LogisticRegression(random_state=42)

# logreg_base = pipeline.fit(X_train, y_train)
# logreg_ypred = logreg_base.predict(X_test)
# logreg_accuracy = accuracy_score(y_test, logreg_ypred)
# logreg_ypred_proba = logreg_base.predict_proba(X_test)
# logreg_logloss = log_loss(y_test, logreg_ypred_proba)

# print(logreg_accuracy)
# print(logreg_logloss)
# print(classification_report(y_test, logreg_ypred))

# ----------------------------------------------------------------------------------------

# preprocessor = ColumnTransformer(
#     transformers = [
#         ('num', StandardScaler(), numerical),
#         ('cat', OrdinalEncoder(), categorical)])

# pipeline = Pipeline(
#     steps = [
#         ('preprocessor', preprocessor),
#         ('model', estimator)])

# ----------------------------------------------------------------------------------------

# df = df[df['year'].notnull()]
# df = df[df['manufacturer'].notnull()]
# df = df[df['model'].notnull()]
# df = df[df['fuel'].notnull()]
# df = df[df['odometer'].notnull()]
# df = df[df['title_status'].notnull()]
# df = df[df['transmission'].notnull()]
# df = df[df['description'].notnull()]
# df = df[df['lat'].notnull()]
# df = df[df['long'].notnull()]
# recent_30 = df['year'].max() - 30
# df_30 = df[df['year'] >= recent_30]
# df_30

In [ ]:
try:
    print('Script executed successfully.')
except:
    print('Failed')